In [1]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [9]:
#read in subject file
with open('target.txt') as f:
     file_ = [line.strip() for line in f]
file_

['have a great day?',
 'is there anything else i can help you with today?',
 'you are welcome.',
 'great.',
 'thank you.',
 'is there anything else i can assist you with?',
 'okay.',
 'is there anything else i can help you with?',
 'what is your account number?',
 'what is your order number?',
 'you are very welcome.',
 'ok great.',
 'is there anything else i can assist you with today?',
 'i am sorry to hear that.',
 'what is your new address?',
 'thanks.',
 'i am happy to look into this for you.',
 'have a safe flight?',
 'thank you for that information.',
 'does that work for you.',
 'does that work.',
 'what is the order number?',
 'his.',
 'ok awesome.',
 'i am happy to assist you.',
 'enjoy the rest of your day.',
 'have a great day further?',
 'no problems.',
 'have you tried reporting the system?',
 'what is your cell number?',
 'i can assist you.',
 'have a wonderful day?',
 'sure.',
 'i can help you.',
 'great news.',
 'have a good day?',
 'i can help you with this.',
 'which 

In [ ]:
'''
preprocessing stage
-------------------
    - spell check
    - lower casing
    - punctuation removal
    - number removal?
    - html removal
'''

def remove_punc(t_string):
    check = lambda char: ' ' if char in set(punctuation) else char
    
    return ''.join(map(check, list(t_string)))

def clean(name):
    #remove spaces
    name = name.strip()
    
    #strip out
    name = name.replace('\"', '')    
    name = encode(name)    
    
    return name

In [38]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def build_vectorizer(corpus, model_type='bag of words', ngrams=1, tokenizer='char'):
    '''
    *add word2vec
    '''
    #think of params
    params = {
        'analyzer': tokenizer,
        'ngram_range' : (1, ngrams)
    }

    if model_type == 'bag of words':
        vectorizer = CountVectorizer(**params)
    elif model_type ==  'tfidf':
        vectorizer = TfidfVectorizer(**params)
    
    
    return vectorizer.fit_transform(corpus), vectorizer 

In [90]:
from sklearn.neighbors import NearestNeighbors

vector_space, vectorizer = build_vectorizer(file_, model_type='tfidf', ngrams=1)
nn = NearestNeighbors(n_jobs=-1, algorithm='brute')
nn.fit(vector_space)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [92]:
print file_[0]
print '*' * 50
new_data_pt=[file_[0]] 
pt=vectorizer.transform(new_data_pt)
# for data_pt in vector_space:
neighbors = nn.radius_neighbors(pt, radius=.25)
indices = neighbors[1][0]
for index in indices:
    print file_[index]

have a great day?
**************************************************
have a great day?
have a a great day?
have great day?
have a great day great?
